# LLM para crear un indicador de calidad

## Primer modelo (4k)

### Introducción

Con la BBDD de Carlos, voy a crear un modelo que, a partir de un indicador de calidad, aprenda a reconocer qué descripciones corresponden a inmuebles de buena calidad. Usaremos un modelo del tipo como una máquina de vectores de soporte (SVM).

In [1]:
import pandas as pd
import numpy as np
import pyreadstat

Importamos el archivo, revisamos las columnas, nos quedamos sólo con las que nos interesan y vemos cuántas filas tenemos.

In [2]:
h=pd.read_spss('rawdata/BDDHabitaclia_4043_join.sav')

In [3]:
print(h.columns.tolist())

['OBJECTID_1', 'codigo_inmueble1', 'Title', 'Type_build', 'Type_opera', 'Link', 'Location', 'Lat_X', 'Lon_Y', 'Climatic_Z', 'Nom_Mun', 'precio_eur', 'superficie', 'superficie_2', 'Unit_price', 'Ln_total_pr', 'Ln_unit_pr', 'numero_habitaciones', 'numero_bano', 'ratio_bano_hab', 'numero_aseo', 'ascensor', 'interac_planta', 'numero_de_piso', 'anyo_constr', 'anyo_constr_ponderad', 'antig_ponderad', 'Inverse_Age', 'Year_Before_1981', 'Year_1982_2006', 'Year_After_2007', 'superficie_terraza_m2', 'grand_terr_20m2', 'superficie_jardin_m2', 'superficie_salon', 'bool_despacho', 'bool_buhardilla', 'bool_trastero', 'bool_lavadero', 'bool_piscina_comunitaria', 'bool_jardin_comunitario', 'bool_amueblado', 'bool_ascensor', 'descripcion', 'bool_aire_acondicionado', 'bool_calefaccion', 'bool_chimenea', 'texto_destacado', 'Description', 'calificacion_consumo_letra', 'calificacion_consumo_valor', 'calificacion_emision_letra', 'calificacion_emision_valor', 'Dum_EPC', 'EPC_A_emision', 'EPC_B_emision', 'EPC

In [7]:
hab=h[['OBJECTID_1', 'codigo_inmueble1', 'precio_eur', 'Unit_price', 'texto_destacado',
         'calidad_cocina', 'diseny_cocina', 'alta_calidad', 'reform_inmob', 'C_contempo', 'C_estado',
         'C_armarios', 'B_contempo', 'B_estado', 'B_lavamano', 'R_contempo', 'R_estado', 
         'R_carpinte', 'R_singular', 'R_ventana']]
hab.head()

,OBJECTID_1,codigo_inmueble1,precio_eur,Unit_price,texto_destacado,calidad_cocina,diseny_cocina,alta_calidad,reform_inmob,C_contempo,C_estado,C_armarios,B_contempo,B_estado,B_lavamano,R_contempo,R_estado,R_carpinte,R_singular,R_ventana
0,1.0,2.872004e+12,345000.0,4011.627907,Unico!!! Dúplex en Collblanc,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
1,2.0,5.020004e+12,370000.0,3814.432990,Piso en venta con amplio balcón junto al CC Gr...,0.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
2,3.0,8.700366e+10,169000.0,2194.805195,TODO EXTERIOR Y REFORMADO,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0
3,4.0,5.000037e+11,229000.0,2410.526316,Piso para entrar a vivir,1.0,0.0,0.0,1.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
4,5.0,5.380035e+11,393000.0,2411.042945,de obra nueva a estrenar. 4hab,1.0,1.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,0.0,1.0


In [8]:
habit=hab[['OBJECTID_1', 'codigo_inmueble1', 'precio_eur', 'Unit_price', 'texto_destacado',
         'alta_calidad']]
habit.head()

,OBJECTID_1,codigo_inmueble1,precio_eur,Unit_price,texto_destacado,alta_calidad
0,1.0,2.872004e+12,345000.0,4011.627907,Unico!!! Dúplex en Collblanc,0.0
1,2.0,5.020004e+12,370000.0,3814.432990,Piso en venta con amplio balcón junto al CC Gr...,1.0
2,3.0,8.700366e+10,169000.0,2194.805195,TODO EXTERIOR Y REFORMADO,1.0
3,4.0,5.000037e+11,229000.0,2410.526316,Piso para entrar a vivir,0.0
4,5.0,5.380035e+11,393000.0,2411.042945,de obra nueva a estrenar. 4hab,0.0


In [9]:
habit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4043 entries, 0 to 4042
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   OBJECTID_1        4043 non-null   float64
 1   codigo_inmueble1  4043 non-null   float64
 2   precio_eur        4043 non-null   float64
 3   Unit_price        4043 non-null   float64
 4   texto_destacado   4043 non-null   object 
 5   alta_calidad      4043 non-null   float64
dtypes: float64(5), object(1)
memory usage: 189.6+ KB


Una de las cosas a las que pueden ser sensibles este tipo de modelos es al desequilibrio entre las clases (por ejemplo, hay muchas más instancias de una clase que de la otra), de manera que su precisión puede ser alta incluso si el modelo predice incorrectamente la clase menos frecuente con frecuencia. Veamos cuántos 0 y 1 tenemos en la columna **alta_calidad**.

In [20]:
conteo = habit['alta_calidad'].value_counts()
print(conteo)

alta_calidad
0.0    3208
1.0     835
Name: count, dtype: int64


Vamos a tener que cuidar cómo se re-distribuyen estos valores cuando elija datos de entrenamiento y testeo.

### Elaboración del modelo

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(habit['texto_destacado'], habit['alta_calidad'], 
                                                    test_size=0.2, random_state=42) #Ojo: elijo 80-20.

# Preprocesamiento y representación de texto
custom_stopwords = ['de', 'la', 'el', 'los', 'las', 'en', 'para', 'con', 'y', 'o', 'un', 'una', 'que', 'se', 'su', 'sus']  # Agrega más palabras si es necesario
tfidf_vectorizer = TfidfVectorizer(stop_words=custom_stopwords)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Entrenar un modelo de clasificación (por ejemplo, SVM)
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_tfidf, y_train)

# Predecir sobre los datos de prueba
y_pred = svm_model.predict(X_test_tfidf)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: {:.2f}%".format(accuracy * 100))

Precisión del modelo: 91.47%


Por si acaso contamos cómo se distribuyeron los 0 y 1 en los datos de entrenamiento. Vemos que es muy similar al dataset original.

In [26]:
conteo2 = y_train.value_counts()
print(conteo2)

alta_calidad
0.0    2573
1.0     661
Name: count, dtype: int64


Algunos indicadores adicionales.

In [23]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calcular precision, recall y F1 sobre los datos de prueba
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Imprimir las métricas
print("Precisión:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Precisión: 0.8387096774193549
Recall: 0.7471264367816092
F1 Score: 0.790273556231003


Algunos apuntes: 

- Precisión (Precision): La precisión se refiere a la proporción de las instancias clasificadas como positivas que son realmente positivas.Se calcula como el número de verdaderos positivos dividido por la suma de verdaderos positivos y falsos positivos. **Es útil cuando el costo de los falsos positivos es alto y deseas minimizarlos**.

- Exactitud (Accuracy): La exactitud es la proporción de todas las predicciones que son correctas. Se calcula como la suma de verdaderos positivos y verdaderos negativos dividido por el total de instancias. Es una medida global del rendimiento del modelo y **es útil cuando todas las clases tienen una importancia similar**.

La precisión del modelo parece ser buena. Veamos qué pasa si comparamos los valores que predice con los valores reales del mismo dataset, pero esta vez completo (100%).

In [12]:
# Predecir sobre los datos de entrenamiento y prueba
y_train_pred = svm_model.predict(X_train_tfidf)
y_test_pred = svm_model.predict(X_test_tfidf)

# Añadir columnas al DataFrame con los valores predichos
habit['alta_calidad_predicha'] = y_train_pred.tolist() + y_test_pred.tolist()

      OBJECTID_1  codigo_inmueble1  precio_eur   Unit_price  \
0            1.0      2.872004e+12    345000.0  4011.627907   
1            2.0      5.020004e+12    370000.0  3814.432990   
2            3.0      8.700366e+10    169000.0  2194.805195   
3            4.0      5.000037e+11    229000.0  2410.526316   
4            5.0      5.380035e+11    393000.0  2411.042945   
...          ...               ...         ...          ...   
4038      4039.0      4.264900e+13    230000.0  4035.087719   
4039      4040.0      4.300000e+13     80000.0  1777.777778   
4040      4041.0      4.425000e+13    159900.0  2050.000000   
4041      4042.0      4.425000e+13    179500.0  2564.285714   
4042      4043.0      4.469700e+13    162000.0  2382.352941   

                                        texto_destacado  alta_calidad  \
0                          Unico!!! Dúplex en Collblanc           0.0   
1     Piso en venta con amplio balcón junto al CC Gr...           1.0   
2                       

C:\Users\rojas\AppData\Local\Temp\ipykernel_35456\425510422.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  habit['alta_calidad_predicha'] = y_train_pred.tolist() + y_test_pred.tolist()


In [13]:
# Contar valores coincidentes entre 'alta_calidad' y 'alta_calidad_predicha'
coincidencias = (habit['alta_calidad'] == habit['alta_calidad_predicha']).sum()

print("Cantidad de valores coincidentes:", coincidencias)

Cantidad de valores coincidentes: 2775


In [14]:
2775/4043

0.6863715063071977

Sólo coincide en el 68%, pero veamos qué pasa cuando lo pasamos a otro dataset.

### Prueba del modelo (6k)

Voy a probar el modelo con la base de datos que me compartió Paúl, que tiene casi 6k observaciones. Primero cargo los datos.

In [16]:
hab6=pd.read_spss('rawdata/BDD 10958 EPC and NO EPC - PEZ_2.sav')
habit6=hab6[['codigo_inmueble1', 'precio_eur', 'Unit_price', 'texto_destacado',
         'alta_calidad']]
habit6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5467 entries, 0 to 5466
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   codigo_inmueble1  5467 non-null   float64
 1   precio_eur        5467 non-null   float64
 2   Unit_price        5467 non-null   float64
 3   texto_destacado   5467 non-null   object 
 4   alta_calidad      5467 non-null   float64
dtypes: float64(4), object(1)
memory usage: 213.7+ KB


Y ahora, con el modelo previamente entrenado, predigo los valores de calidad y los comparo con los valores reales.

In [17]:
# Preprocesamiento del texto en 'texto_destacado' usando el mismo TfidfVectorizer
X_habit6_tfidf = tfidf_vectorizer.transform(habit6['texto_destacado'])

# Predecir la calidad de los inmuebles en 'habit6' utilizando el modelo ya entrenado
y_habit6_pred = svm_model.predict(X_habit6_tfidf)

# Agregar una columna 'alta_calidad_predicha' al DataFrame 'habit6' con las predicciones del modelo
habit6['alta_calidad_predicha'] = y_habit6_pred

coincidencias6 = (habit6['alta_calidad'] == habit6['alta_calidad_predicha']).sum()

print("Cantidad de valores coincidentes:", coincidencias6)

Cantidad de valores coincidentes: 4948


C:\Users\rojas\AppData\Local\Temp\ipykernel_35456\1795496055.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  habit6['alta_calidad_predicha'] = y_habit6_pred


In [18]:
4948/5467

0.9050667642216939

Esta vez sí se acerca bastante al valor de exactitud que indicaba el modelo.

In [28]:
from joblib import dump, load

# Guardar el modelo entrenado
dump(svm_model, 'modelos/LLM1-SVM.joblib')

# Cargar el modelo entrenado
#svm_model_cargado = load('modelo_svm.joblib')

# Ahora puedes usar svm_model_cargado para hacer predicciones como lo harías con svm_model


['modelos/LLM1-SVM.joblib']

## Segundo modelo (4k)

Esta vez usaremos una red neuronal con tres capas densas, cada una seguida de una capa de abandono (dropout) para evitar el sobreajuste. Utilizamos la función de activación 'relu' en las capas ocultas y 'sigmoid' en la capa de salida para problemas de clasificación binaria. El optimizador Adam se utiliza para minimizar la pérdida de entropía cruzada binaria, y la exactitud (*accuracy*) se utiliza como métrica de evaluación.

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

##### Esto ya lo tengo del modelo anterior, por eso no lo ejecuto
# Dividir los datos en características (X) y etiquetas (y)
#X = habit['texto_destacado']
#y = habit['alta_calidad']

# Dividir los datos en conjuntos de entrenamiento y prueba
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Representación TF-IDF de las características de texto
#tfidf_vectorizer = TfidfVectorizer(stop_words=custom_stopwords)
#X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
#X_test_tfidf = tfidf_vectorizer.transform(X_test)

##### A partir de aquí ya son cosas nuevas para este modelo de redes neuronales
# Escalar las características numéricas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_tfidf.toarray())
X_test_scaled = scaler.transform(X_test_tfidf.toarray())

# Crear el modelo de redes neuronales
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid') #Sigmoid es más adecuado para variables binarias
])

# Compilar el modelo
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test))

# Evaluar el modelo en los datos de prueba
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Exactitud del modelo en los datos de prueba:", accuracy)


Epoch 1/10



102/102 [==============================] - 2s 7ms/step - loss: 0.6758 - accuracy: 0.7365 - val_loss: 0.4593 - val_accuracy: 0.7849
Epoch 2/10
102/102 [==============================] - 0s 4ms/step - loss: 0.5219 - accuracy: 0.7870 - val_loss: 0.3740 - val_accuracy: 0.8232
Epoch 3/10
102/102 [==============================] - 0s 5ms/step - loss: 0.3894 - accuracy: 0.8411 - val_loss: 0.3401 - val_accuracy: 0.8430
Epoch 4/10
102/102 [==============================] - 0s 4ms/step - loss: 0.2799 - accuracy: 0.8782 - val_loss: 0.3446 - val_accuracy: 0.8616
Epoch 5/10
102/102 [==============================] - 0s 4ms/step - loss: 0.2361 - accuracy: 0.9051 - val_loss: 0.3510 - val_accuracy: 0.8727
Epoch 6/10
102/102 [==============================] - 0s 4ms/step - loss: 0.1980 - accuracy: 0.9236 - val_loss: 0.3701 - val_accuracy: 0.8789
Epoch 7/10
102/102 [==============================] - 0s 4ms/step - loss: 0.1543 - accuracy: 0.9378 - val_loss: 0.3885 - val_accuracy: 0.8789
Epoch 8/10
102/10

Exactitud ligeramente menor que el modelo anterior.

In [35]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

# Calcular las predicciones binarias del modelo en el conjunto de datos de prueba
umbral = 0.5  # Umbral para convertir las probabilidades en etiquetas binarias
y_pred_binario = [1 if pred >= umbral else 0 for pred in model.predict(X_test_scaled)]

# Calcular las métricas de evaluación
accuracy = accuracy_score(y_test, y_pred_binario)
recall = recall_score(y_test, y_pred_binario)
precision = precision_score(y_test, y_pred_binario)
f1 = f1_score(y_test, y_pred_binario)
conf_matrix = confusion_matrix(y_test, y_pred_binario)

# Imprimir las métricas
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)

26/26 [==============================] - 0s 2ms/step
Accuracy: 0.8702101359703337
Recall: 0.6436781609195402
Precision: 0.7225806451612903
F1 Score: 0.6808510638297872
Confusion Matrix:
 [[592  43]
 [ 62 112]]


- Precisión (Accuracy):
    - La precisión es la proporción de predicciones correctas sobre el total de predicciones.
    - Una precisión del 1.0 indica que todas las predicciones son correctas, mientras que una precisión del 0.0 indica que ninguna predicción es correcta.
    - Es una métrica general del rendimiento del modelo, pero puede ser engañosa si hay un desequilibrio en las clases objetivo.

- Recall (Exhaustividad):
    - La exhaustividad es la proporción de positivos reales que se identificaron correctamente.
    - Una exhaustividad del 1.0 indica que todas las instancias positivas se han identificado correctamente, mientras que una exhaustividad del 0.0 indica que ninguna instancia positiva se ha identificado correctamente.
    - Es útil cuando la identificación de instancias positivas es crítica y no se pueden permitir falsos negativos.

- Precisión (Precision):
    - La precisión es la proporción de instancias positivas predichas que fueron correctamente identificadas.
    - Una precisión del 1.0 indica que todas las instancias predichas como positivas son verdaderas positivas, mientras que una precisión del 0.0 indica que ninguna instancia predicha como positiva es realmente positiva.
    - Es útil cuando es importante evitar falsos positivos.

- F1 Score:
    - El puntaje F1 es la media armónica de precisión y exhaustividad.
    - Proporciona un equilibrio entre precisión y exhaustividad, lo que lo hace útil cuando se desea tener un buen rendimiento en ambas métricas.
    - Un puntaje F1 del 1.0 indica un equilibrio perfecto entre precisión y exhaustividad.

- Matriz de Confusión:
    - La matriz de confusión es una tabla que describe la calidad de las predicciones del modelo.
    - Proporciona una descripción detallada de los resultados de clasificación, mostrando el número de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos.
    - Es útil para identificar dónde el modelo está cometiendo errores y para evaluar el desempeño en cada clase por separado.

### Prueba del modelo (6K)

In [32]:

# Preprocesamiento del texto en 'texto_destacado' utilizando el mismo TfidfVectorizer y StandardScaler
#X_habit6_tfidf = tfidf_vectorizer.transform(habit6['texto_destacado']) #### Esto ya no lo ejecuto porque ya lo hice con el modelo anterior
X_habit6_scaled = scaler.transform(X_habit6_tfidf.toarray())

#### A partir de aquí ya es nuevo
# Predecir la calidad de los inmuebles en 'habit6' utilizando el modelo de redes neuronales ya entrenado
y_habit6_pred = model.predict(X_habit6_scaled)

# Convertir las predicciones a etiquetas binarias (0 o 1) utilizando un umbral (por ejemplo, 0.5)
umbral = 0.5
acp_LLM_ANN = [1 if pred >= umbral else 0 for pred in y_habit6_pred] #acp de alta_calidad_predicha

# Agregar una columna 'alta_calidad_predicha' al DataFrame 'habit6' con las predicciones del modelo
habit6['acp-LLM-ANN'] = acp_LLM_ANN

#coincidencias6 = (habit6['alta_calidad'] == habit6['acp-LLM-ANN']).sum()

print("Cantidad de valores coincidentes:", (habit6['alta_calidad'] == habit6['acp-LLM-ANN']).sum())


171/171 [==============================] - 1s 2ms/step
Cantidad de valores coincidentes: 4869


C:\Users\rojas\AppData\Local\Temp\ipykernel_35456\1738249818.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  habit6['acp-LLM-ANN'] = acp_LLM_ANN


In [33]:
4869/5467

0.8906164258276934

Parece tener un rendimiento ligeramente menor que el **modelo SVM**. Guardamos los resultados.

In [36]:
# Guardar el modelo entrenado
model.save('modelos/LLM1-ANN')

# Guardar el historial de entrenamiento
dump(history, 'modelos/LLM1-ANN/history.joblib')

INFO:tensorflow:Assets written to: modelos/LLM1-ANN\assets


INFO:tensorflow:Assets written to: modelos/LLM1-ANN\assets


['modelos/LLM1-ANN/history.joblib']

## Tercer modelo (4k)

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

#### Esto ya lo tengo

# Dividir los datos en características (X) y etiquetas (y)
#X = habit['texto_destacado']
#y = habit['alta_calidad']

# Dividir los datos en conjuntos de entrenamiento y prueba
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Representación TF-IDF de las características de texto
#tfidf_vectorizer = TfidfVectorizer(stop_words=custom_stopwords)
#X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
#X_test_tfidf = tfidf_vectorizer.transform(X_test)

#### Esto ya es nuevo
# Crear y entrenar el modelo de Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train_tfidf, y_train)

# Predecir las etiquetas en el conjunto de prueba
y_pred = random_forest.predict(X_test_tfidf)

# Calcular las métricas de evaluación
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Imprimir las métricas
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.9048207663782447
Recall: 0.6954022988505747
Precision: 0.8344827586206897
F1 Score: 0.7586206896551724
Confusion Matrix:
 [[611  24]
 [ 53 121]]


### Prueba del modelo (6k)

In [39]:
# import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Representación TF-IDF de las características de texto
#tfidf_vectorizer = TfidfVectorizer(stop_words=custom_stopwords)
#X_habit6_tfidf = tfidf_vectorizer.fit_transform(habit6['texto_destacado'])

# Predecir las etiquetas en el conjunto de datos habit6
habit6['acp-LLM-RF'] = random_forest.predict(X_habit6_tfidf)

print("Cantidad de valores coincidentes:", (habit6['alta_calidad'] == habit6['acp-LLM-RF']).sum())

Cantidad de valores coincidentes: 5045


C:\Users\rojas\AppData\Local\Temp\ipykernel_35456\1587127615.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  habit6['acp-LLM-RF'] = random_forest.predict(X_habit6_tfidf)


In [40]:
5045/5467

0.9228095847814157

De momento, el mejor modelo.

In [41]:
#from joblib import dump #Ya lo había cargado antes

# Guardar el modelo entrenado
dump(random_forest, 'modelos/LLM1-RF.joblib')

['modelos/LLM1-RF.joblib']

Lo que puedo hacer es trabajar con una BBDD más grande: Pedir al CPSV los datasets con datos sobre calidad arquitectónica de todos los años, juntarlos, y con eso entrenar un nuevo modelo más grande.

La otra opción, más tediosa, es utilizar un LLM pre-entrenado y hacerle fine tuning con mis datos, pero eso aún no he descubierto como hacerlo bien.